<a href="https://colab.research.google.com/github/harish-kumar-kp/LLM-Doubt/blob/main/CasedDisBert_CustomSquad_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os

In [10]:
"""https://huggingface.co/distilbert-base-uncased-distilled-squad"""
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "distilbert-base-cased-distilled-squad"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is p

In [11]:
res # score == F1 Score: it's computed on the individual words in the prediction vs the true words provided in context

{'score': 0.2985280752182007,
 'start': 59,
 'end': 132,
 'answer': 'gives freedom to the user and let people easily switch between frameworks'}

In [12]:
model

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

\\# How to fine-tune a QA Model?

In [13]:
pip install datasets

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
main_dir = "/content/gdrive/MyDrive/QADistilBert"
os.chdir(main_dir)
os.listdir()

['old',
 'dataset',
 'codes',
 'train_squad_dataset.json',
 'valid_squad_dataset.json',
 'model',
 'wandb',
 'test-trainer',
 '.gradio']

In [16]:
from datasets import load_dataset
data_files  = {"train": "dataset/squad_01/train_customSquad-v1.1.json", "validation": "dataset/squad_01/validation_customSquad-v1.1.json"}

In [17]:
csquad = load_dataset("json", data_files=data_files, field="data")
csquad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 75
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 50
    })
})

In [18]:
csquad['validation'][0]

{'id': 'i76',
 'title': 'Futuristic',
 'context': 'AGI refers to AI systems that can perform any intellectual task that humans can do, surpassing current AI limited to specific tasks. Unlike traditional AI, AGI could comprehend, learn, and apply knowledge across various domains without needing task-specific programming. The implications are vast, from revolutionizing industries and economies to transforming fields like medicine, education, and research. AGI would enable machines to innovate, solve complex problems, and adapt autonomously. Ethical considerations, such as control and safety, are central, as AGI could lead to unpredictable impacts. Safeguards and regulations are necessary to prevent unintended consequences, such as job displacement or AI-driven decision-making biases. If managed responsibly, AGI could improve global welfare by solving critical issues, such as climate change, resource scarcity, and disease eradication, faster than humans can today.',
 'question': 'What is 

In [19]:
# Preprocess the data to a BERT format
def preprocess_function(examples):
    """Courtesy of https://huggingface.co/docs/transformers/tasks/question_answering"""
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [20]:
tokenized_squad = csquad.map(preprocess_function, batched=True, remove_columns=csquad["train"].column_names)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
#used for Large Squad Datasets
#small_train_dataset = tokenized_squad["train"].shuffle(seed=42).select(range(100))
#small_eval_dataset = tokenized_squad["validation"].shuffle(seed=42).select(range(100))

In [21]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [22]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_name) # remember that model_name is "distilbert-base-uncased-distilled-squad"

In [23]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [24]:
pip install evaluate

In [25]:
import numpy as np
import evaluate

In [26]:
metric = evaluate.load("accuracy")

In [27]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# customised training parameters
'''
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=0.1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01

)
'''

In [28]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: harishk-kotte (harishk-kotte-guvi). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,1.362060
2,No log,1.236961
3,No log,1.229565


TrainOutput(global_step=30, training_loss=1.4775936126708984, metrics={'train_runtime': 29.9638, 'train_samples_per_second': 7.509, 'train_steps_per_second': 1.001, 'total_flos': 22047710630400.0, 'train_loss': 1.4775936126708984, 'epoch': 3.0})

wandb API Key : cc4856cc44f1dae7844c269adf899daa328e7a14

*   List item
*   List item



---



*   List item
*   List item



In [29]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [30]:
main_dir = "/content/gdrive/MyDrive/QADistilBert"
os.chdir(main_dir)
os.listdir()

['old',
 'dataset',
 'codes',
 'train_squad_dataset.json',
 'valid_squad_dataset.json',
 'model',
 'wandb',
 'test-trainer',
 '.gradio']

In [31]:
trainer.save_model("model/save/customTrained_Distilbert_Squad")
trainer.evaluate()

{'eval_loss': 1.2295647859573364,
 'eval_runtime': 0.7985,
 'eval_samples_per_second': 62.62,
 'eval_steps_per_second': 8.767,
 'epoch': 3.0}

In [ ]:
#torch.save(model.state_dict(), "simple_distilbert_qa.model")

In [32]:
model.save_pretrained("model/distilbert-finetuned-squad")
tokenizer.save_pretrained("model/distilbert-finetuned-squad")


('model/distilbert-finetuned-squad/tokenizer_config.json',
 'model/distilbert-finetuned-squad/special_tokens_map.json',
 'model/distilbert-finetuned-squad/vocab.txt',
 'model/distilbert-finetuned-squad/added_tokens.json',
 'model/distilbert-finetuned-squad/tokenizer.json')

In [33]:
import os
import torch
from transformers import pipeline , AutoTokenizer ,AutoModelForQuestionAnswering
from transformers import DistilBertConfig
from transformers import DistilBertForMaskedLM

In [3]:
#model.load_state_dict(torch.load("simple_distilbert_qa.model"))

In [34]:
# Path to your custom-trained model
model_path = "model/save/customTrained_Distilbert_Squad"

# Load the tokenizer and model
bert_model = AutoModelForQuestionAnswering.from_pretrained(model_path)
bert_tokenizer = AutoTokenizer.from_pretrained(model_path)

'''
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)
'''

'\ntokenizer = DistilBertTokenizer.from_pretrained(model_path)\nmodel = DistilBertForSequenceClassification.from_pretrained(model_path)\n'

In [35]:
qa_pipeline = pipeline('question-answering', model=model_path,tokenizer=model_path)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [36]:
context = """
In this sequel set eleven years after "The Terminator," young John Connor (Edward Furlong), the key to civilization's victory over a future robot uprising,
is the target of the shape-shifting T-1000 (Robert Patrick), a Terminator sent from the future to kill him. Another Terminator, the revamped T-800
(Arnold Schwarzenegger), has been sent back to protect the boy. As John and his mother (Linda Hamilton) go on the run with the T800,
the boy forms an unexpected bond with the robot.
"""
question = "Who is young John Connor?"

In [37]:
result = qa_pipeline({
    'question': question,
    'context': context
})

print(result)

{'score': 0.5112108588218689, 'start': 76, 'end': 90, 'answer': 'Edward Furlong'}


In [38]:
import string
import re
from collections import Counter

# Function to normalize answers (remove articles, punctuation, etc.)
def normalize_answer(s):
    """Lowercase, remove punctuation, articles, and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punctuation(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punctuation(lower(s))))

# Exact Match calculation
def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)

# F1 Score calculation
def f1_score(prediction, ground_truth):
    pred_tokens = normalize_answer(prediction).split()
    truth_tokens = normalize_answer(ground_truth).split()

    common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
    num_common = sum(common_tokens.values())

    if num_common == 0:
        return 0.0

    precision = num_common / len(pred_tokens)
    recall = num_common / len(truth_tokens)

    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [39]:
#from datasets import load_dataset

# Load the SQuAD dataset
dataset = load_dataset("squad", split="validation[:100]")  # Load a small subset for testing
#dataset = load_dataset("csquad")  # Load a small subset for testing
# Initialize scores
total_em, total_f1 = 0, 0

# Iterate over the dataset and compute metrics for each example
for example in dataset:
    question = example['question']
    context = example['context']
    ground_truth = example['answers']['text'][0]  # Ground truth answer (first one if multiple)

    # Get model's prediction
    predicted_result = qa_pipeline({
        'question': question,
        'context': context
    })
    predicted_answer = predicted_result['answer']

    # Compute EM and F1 for this example
    em = exact_match_score(predicted_answer, ground_truth)
    f1 = f1_score(predicted_answer, ground_truth)

    total_em += em
    total_f1 += f1

# Average over the dataset
num_examples = len(dataset)
average_em = total_em / num_examples
average_f1 = total_f1 / num_examples

print(f"Average Exact Match: {average_em}")
print(f"Average F1 Score: {average_f1}")

Average Exact Match: 0.58
Average F1 Score: 0.7105238095238094


In [40]:
def EM_ScoreF1(context,question,goldAnswer=""):
  # Perform question-answering
  predicted_result = qa_pipeline({
      'question': question,
      'context': context
  })
  # Ground truth (the correct answer)
  if goldAnswer=="":
    ground_truth = "Answer Unavailable"
  else:
    ground_truth = goldAnswer

  # Get the predicted answer
  predicted_answer = predicted_result['answer']
  # Compute Exact Match and F1 Score
  em_score = exact_match_score(predicted_answer, ground_truth)
  f1 = f1_score(predicted_answer, ground_truth)
  return(f"Machine Answer: {predicted_result['answer']}"+" Vs 'Human Answer':"+ground_truth), (f"Exact Match: {em_score}"), (f"F1 Score: {f1}")

In [41]:
index = 4

In [42]:
csquad['validation'][index]

{'id': 'i80',
 'title': 'Futuristic',
 'context': 'AGI refers to AI systems that can perform any intellectual task that humans can do, surpassing current AI limited to specific tasks. Unlike traditional AI, AGI could comprehend, learn, and apply knowledge across various domains without needing task-specific programming. The implications are vast, from revolutionizing industries and economies to transforming fields like medicine, education, and research. AGI would enable machines to innovate, solve complex problems, and adapt autonomously. Ethical considerations, such as control and safety, are central, as AGI could lead to unpredictable impacts. Safeguards and regulations are necessary to prevent unintended consequences, such as job displacement or AI-driven decision-making biases. If managed responsibly, AGI could improve global welfare by solving critical issues, such as climate change, resource scarcity, and disease eradication, faster than humans can today.',
 'question': 'What are

In [43]:
# Example context and question

context_var =csquad['validation'][index]['context']
question_var =csquad['validation'][index]['question']
goldAnswer_var = csquad['validation'][index]['answers']['text'][0]


In [44]:
print(question_var)

What are the unintended consequences?


In [45]:
print(EM_ScoreF1(context_var,question_var,goldAnswer_var))

("Machine Answer: job displacement or AI-driven decision-making biases Vs 'Human Answer':job displacement or AI-driven decision-making biases", 'Exact Match: True', 'F1 Score: 1.0')


In [42]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


#Gradio App To Check EM and F1 score with compared Machine and Human Answers.

In [46]:
import gradio as gr
def EM_ScoreF1(context,question,goldAnswer=""):
  # Perform question-answering
  predicted_result = qa_pipeline({
      'question': question,
      'context': context
  })
  # Ground truth (the correct answer)
  if goldAnswer=="":
    ground_truth = "Answer Unavailable"
  else:
    ground_truth = goldAnswer
  # Get the predicted answer
  predicted_answer = predicted_result['answer']
  # Compute Exact Match and F1 Score
  em_score = exact_match_score(predicted_answer, ground_truth)
  f1 = f1_score(predicted_answer, predicted_answer)
  return(f"'Machine Answer': {predicted_result['answer']}"+"   Vs  'Human Answer':"+ground_truth), (f"Exact Match: {em_score}"), (f"F1 Score: {f1}")

demo = gr.Interface(
    fn=EM_ScoreF1,
    inputs=["text", "text","text"],
    outputs=["text","text","text"],
)

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1fcc1db4350b979279.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:



# Perform question-answering
predicted_result = qa_pipeline({
    'question': question,
    'context': context
})

# Ground truth (the correct answer)
ground_truth = csquad['validation'][index]['answers']['text'][0]

# Get the predicted answer
predicted_answer = predicted_result['answer']

# Compute Exact Match and F1 Score
em_score = exact_match_score(predicted_answer, ground_truth)
f1 = f1_score(predicted_answer, ground_truth)

print(f"Exact Match: {em_score}")
print(f"F1 Score: {f1}")

Exact Match: True
F1 Score: 1.0
